In [26]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
    Union,
    List
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.prebuilt import ToolNode
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_ollama import ChatOllama
from langchain_core.messages import ToolMessage,SystemMessage
from langchain_core.runnables import RunnableConfig
from pydantic import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import json

class VerifierBase(BaseModel):
    step: str = Field(..., description="The step as given in the experiment.")
    search_query: str = Field(..., description="The phrase or keywords you would search on the web to verify the step.")
    verification: str = Field(..., description="The expected correct verification result based on technical knowledge and search results.")

class VerifierStructure(BaseModel):
    steps: List[VerifierBase] = Field(..., description="A list of verification steps, each containing step details, search query, and verification result.")

class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]
    generated_exps: dict
    exp_title:str
    reward:Union[int,float]
    verified_structure: list
    tool_results : dict

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wrapper = DuckDuckGoSearchAPIWrapper(max_results=3)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format='list')

tools = [search,wikipedia]
# model = ChatOllama(model="qwq",base_url="192.168.23.138:11439")
# model = ChatOllama(model="jacob-ebey/phi4-tools:latest",base_url="192.168.23.138:11439")
model = ChatOllama(model="qwen2.5:32b",base_url="192.168.23.138:11439")
tool_model = model.bind_tools(tools)
verifier_model = model.with_structured_output(VerifierStructure,method='json_schema')

In [27]:
verifier_system_prompt = '''You are a Verifier AI responsible for checking the accuracy of experimental steps across various scientific and technical domains. Your task is to verify whether each step is correct by using reasonable web searches.

For each step in the experiment, you will:

Identify the key concept(s) involved.
Generate an appropriate search query to verify the correctness of the step.
Validate the step by using the search results to confirm or correct the instruction.
Your output should be structured in JSON format with the following fields:

"step": The step as given in the experiment.
"search_query": The phrase or keywords you would search on the web to verify the step.
"verification": The expected correct verification result based on technical knowledge and search results.
Be precise, factual, and ensure that the verification aligns with established scientific and technical principles'''

In [28]:
def seed_generator(state:AgentState):
    exp_generated = model.invoke(state['messages'])
    return {"generated_exps":{state["exp_title"]:exp_generated.content}}

def verifier_generator(state:AgentState):
    system_prompt = SystemMessage(verifier_system_prompt)
    struct = verifier_model.invoke([system_prompt]+[str(state["generated_exps"])])
    # print(struct.steps)
    
    return {"verified_structure":struct.steps}
tools_by_name = {tool.name: tool for tool in tools}
def tool_decider_node(state:AgentState):
    system_prompt = '''you are given the following 
    1. step: which a step from the experiment
    2. search_query: the query which you have to search 
    you have to make a tool call or tool calls(multiple) which might give the best possible answer try to use all tools which are provided'''
    all_step_tool_results =[]
    for s in state['verified_structure']:
        query = dict(s)
        query.pop('verification')
        tool_results = {"step": query["step"]}
        resp = tool_model.invoke([SystemMessage(system_prompt)]+ [str(query)])
        all_step_tool_results.append(tool_results)
        for tool_call in resp.tool_calls:
            tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            if tool_call["name"] == 'duckduckgo_results_json':
                
                snippets = []
                for snippet in tool_result:
                    snippets.append(snippet['snippet'])
                tool_results.update({"duckduckgo": snippets})
            elif tool_call["name"] == "wikipedia":
                # print(type(tool_result))
                tool_results.update({"wiki": tool_result})
            # print(tool_result)
        
    return{"tool_results":all_step_tool_results}

In [29]:
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)
workflow.add_node("generator",seed_generator)
workflow.add_node("verifier",verifier_generator)
workflow.add_node("toolcaller",tool_decider_node)
workflow.set_entry_point("generator")
workflow.set_finish_point("toolcaller")
workflow.add_edge("generator","verifier")
workflow.add_edge("verifier","toolcaller")
graph = workflow.compile()

In [30]:
exp = "To set up and test a unity gain amplifier using an IC 741 operational amplifier."

query = f"what should be the required materials and experiment procedure of the following lab Experiment ({exp})?"
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", query)], "exp_title":exp,"verified_structure":[]}
final = graph.invoke(inputs)
# print_stream(graph.stream(inputs, stream_mode="values"))

c:\Users\rahul\OneDrive\Documents\Python Scripts\LLM-Agents\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\rahul\OneDrive\Documents\Python Scripts\LLM-Agents\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [39]:
for i in final['tool_results']:
    if 'wiki' in i.keys():
        i.pop('wiki')
    print(i)

{'step': 'Place the 741 operational amplifier on a breadboard.', 'duckduckgo': ['Place the IC 741 on the breadboard. Connect a resistor from the input signal to the inverting input (pin 2). Connect a capacitor (pin 6) between the inverting input and output. Connect the non-inverting input to the ground (pin 3). Make sure the power connections are: pin 4 to -15V, pin 7 to +15V. Use a signal generator to apply a sine wave (1 ...', 'Ãƒâ€šÃ‚Â· Pin diagram of the IC 741. Ãƒâ€šÃ‚Â· The resistor and capacitor values available on the board to be used in the circuit design. ... 3. Place all the components on the breadboard. IC 741, resistor, capacitor on the virtual breadboard. 4. ... 8. After verifying the circuit using verify icon on breadboard, lock the ...', '9) 741 Touch Switch Circuit. This IC 741 touch switch circuit a sensitive touch-operated switch, which could also be applied (if you like) like a rain detector.. As shown in the figure Pin 3 of the IC 741 is fixed by resistors R1 and R

In [33]:
len([dict(i) for i in final['verified_structure']])

9

In [34]:
md_content = final['generated_exps'][exp]

In [35]:
from IPython.display import display, Markdown
display(Markdown(md_content))

Setting up and testing a unity gain amplifier using an IC 741 operational amplifier is a fundamental electronics laboratory experiment. This experiment demonstrates the concept of voltage follower, which is a configuration where the output voltage equals the input voltage, hence the term "unity gain."

### Required Materials:
1. **IC 741 Operational Amplifier**
2. **DC Power Supply** - to power the IC 741
3. **Function Generator or Signal Source** - to provide an input signal for testing the amplifier
4. **Oscilloscope** - to observe and measure both input and output signals
5. **Breadboard** and connecting wires
6. **Resistors**:
   - 2 x 10kΩ resistors (for biasing the IC, if needed)
7. **Capacitors**: 
   - Decoupling capacitors (e.g., 10µF) to stabilize power supply inputs

### Experiment Procedure:

#### Step 1: Circuit Setup
1. Place the 741 operational amplifier on a breadboard.
2. Connect pin 4 (V-) of the op-amp to ground. This is typically done in all op-amp configurations.
3. Connect pin 8 (V+) and pin 7 (V-) of the op-amp to your DC power supply. Typically, you would connect V+ to +15V and V- to -15V or use a single-ended positive voltage with ground for V- if that's what your setup allows.
4. Connect the input signal from your function generator to pin 2 (V+) of the op-amp.
5. Connect pin 3 (output) directly back to pin 2 (inverting input). This direct connection is how we achieve unity gain, as there’s no feedback resistor or any other components that would modify the ratio between the input and output.

#### Step 2: Decoupling Capacitors
1. Add decoupling capacitors across each power supply line and ground to stabilize voltage levels at the op-amp pins (between V+ and GND, and V- and GND).

#### Step 3: Testing the Unity Gain Amplifier
1. Turn on your DC power supply.
2. Apply a signal from your function generator to pin 2 of the IC.
3. Observe both input and output signals using an oscilloscope:
   - Connect one channel of the oscilloscope to the point where you apply the signal (input).
   - Connect another channel to the output (pin 3) of the op-amp.
4. Adjust the function generator settings such as frequency, amplitude, and offset if needed to observe the behavior of your amplifier under different conditions.

#### Step 4: Measurement and Analysis
1. Measure both input and output voltages using the oscilloscope to ensure they match (within the range allowed by the op-amp's specifications).
2. Analyze waveforms for any distortion or changes in amplitude.
3. Record observations about how well the amplifier maintains unity gain under different signal conditions.

#### Step 5: Safety Precautions
1. Ensure that all connections are secure to prevent short circuits.
2. Use appropriate power supply voltages; typically, ±15V is safe for an op-amp like the IC 741.
3. Be cautious when handling electrical components and always ensure the circuit is powered off before making any physical changes.

This experiment provides a basic understanding of operational amplifiers in unity gain configuration and serves as a foundation to explore more complex applications such as non-inverting, inverting amplifiers, or integrators/differentiators.